In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [37]:
df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('test_kaggle.csv')
result = df_test['id']
result = pd.DataFrame(result)

In [45]:
n = df_test.nunique(axis=0)
n

id               5970
Store_id          110
SKU_id              2
Date               28
Promo               1
Demand              0
Regular_Price      11
Promo_Price         4
dtype: int64

In [32]:
df_test.describe()

,id,Store_id,SKU_id,Promo,Demand,Regular_Price,Promo_Price
count,5970.000000,5970.000000,5970.000000,1514.0,0.0,5970.000000,1514.000000
mean,2984.500000,53.849246,1.500000,1.0,NaN,135.004390,119.452272
std,1723.534885,30.853390,0.500042,0.0,NaN,3.490976,5.494316
min,0.000000,1.000000,1.000000,1.0,NaN,128.980000,114.170000
25%,1492.250000,27.000000,1.000000,1.0,NaN,131.700000,114.170000
50%,2984.500000,54.000000,1.500000,1.0,NaN,134.960000,119.600000
75%,4476.750000,80.000000,2.000000,1.0,NaN,138.500000,125.040000
max,5969.000000,110.000000,2.000000,1.0,NaN,145.560000,127.760000


### NaN reconstructing

In [33]:
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_train['Year'] = df_train['Date'].apply(lambda time: time.year)
df_train['Month'] = df_train['Date'].apply(lambda time: time.month)
df_train['Day'] = df_train['Date'].apply(lambda time: time.day)
df_train.drop(['Date'], axis=1, inplace=True)
df_train['Promo'].fillna(0, inplace=True)
df_train['Promo_Price'].fillna(df_train['Regular_Price'], inplace=True)

df_test['Date'] = pd.to_datetime(df_test['Date'])
df_test['Year'] = df_test['Date'].apply(lambda time: time.year)
df_test['Month'] = df_test['Date'].apply(lambda time: time.month)
df_test['Day'] = df_test['Date'].apply(lambda time: time.day)
df_test.drop(['Demand', 'id', 'Date'], axis=1, inplace=True)
df_test['Promo'].fillna(0, inplace=True)
df_test['Promo_Price'].fillna(df_test['Regular_Price'], inplace=True)

### Test and val split

In [34]:
df_train

,Store_id,SKU_id,Promo,Demand,Regular_Price,Promo_Price,Year,Month,Day
0,1,1,0.0,22,163.78,163.78,2015,1,1
1,1,1,0.0,41,163.78,163.78,2015,2,1
2,1,1,0.0,35,163.78,163.78,2015,3,1
3,1,1,0.0,72,163.78,163.78,2015,4,1
4,1,1,0.0,25,163.78,163.78,2015,5,1
...,...,...,...,...,...,...,...,...,...
86911,106,2,0.0,9,138.50,138.50,2016,5,18
86912,106,2,0.0,3,138.50,138.50,2016,5,19
86913,106,2,0.0,13,138.50,138.50,2016,5,20
86914,106,2,0.0,3,138.50,138.50,2016,5,21


In [35]:
X_train = df_train.drop(['Demand'], axis=1)
y_train = df_train['Demand']
X_test = df_test

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [36]:
scaler = StandardScaler()

X_train[['Store_id', 'Regular_Price', 'Promo_Price', 'Month', 'Day']] = scaler.fit_transform(X_train[['Store_id', 'Regular_Price', 'Promo_Price', 'Month', 'Day']])
X_val[['Store_id', 'Regular_Price', 'Promo_Price', 'Month', 'Day']] = scaler.transform(X_val[['Store_id', 'Regular_Price', 'Promo_Price', 'Month', 'Day']])
X_test[['Store_id', 'Regular_Price', 'Promo_Price', 'Month', 'Day']] = scaler.transform(X_test[['Store_id', 'Regular_Price', 'Promo_Price', 'Month', 'Day']])

X_train['Year'] = X_train['Year'] - 2015
X_val['Year'] = X_val['Year'] - 2015
X_test['Year'] = X_test['Year'] - 2015

X_train[['Promo']] = X_train[['Promo']].astype(int)
X_val[['Promo']] = X_val[['Promo']].astype(int)
X_test[['Promo']] = X_test[['Promo']].astype(int)

X_train

,Store_id,SKU_id,Promo,Regular_Price,Promo_Price,Year,Month,Day
38327,1.311977,1,1,-0.817720,-1.373594,0,0.013039,-0.125784
24627,0.194117,1,0,-0.166671,0.122405,0,-1.412918,-0.673702
68925,0.232664,2,0,0.513773,0.631942,0,1.153804,1.408387
84058,1.543259,2,0,-0.092639,0.177843,0,1.438996,1.627555
50839,-1.155025,2,0,-0.077397,0.189256,1,-0.272152,0.202967
...,...,...,...,...,...,...,...,...
6265,-1.193571,1,0,-1.202035,-0.652906,0,0.013039,0.422135
54886,-0.846649,2,0,-0.077397,0.189256,1,-0.557344,1.627555
76820,0.849415,2,0,-0.077397,0.189256,1,1.438996,-1.221620
860,-1.617587,1,0,-0.965785,-0.475995,0,1.724187,0.422135


In [ ]:
#make one hot encoding for Store_id
X_train = pd.get_dummies(X_train, columns=['Store_id'])
X_val = pd.get_dummies(X_val, columns=['Store_id'])
X_test = pd.get_dummies(X_test, columns=['Store_id'])

### LR

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print('MAE: ', mean_absolute_error(y_val, lr.predict(X_val)), 'RMSE: ', np.sqrt(mean_squared_error(y_val, lr.predict(X_val))))

MAE:  136.6038475309344 RMSE:  285.7253936472334


### Random forest

In [10]:
from sklearn.ensemble import RandomForestRegressor

num = [10, 20, 50, 100, 300]

for i in num:
    rf = RandomForestRegressor(n_estimators=i, max_depth=10)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    print(i, 'MAE: ', mean_absolute_error(y_val, rf.predict(X_val)), 'RMSE: ', np.sqrt(mean_squared_error(y_val, rf.predict(X_val))))

10 MAE:  84.24326763673713 RMSE:  221.55850581508807
20 MAE:  84.04279012077706 RMSE:  219.95753371498597
50 MAE:  83.60708195342973 RMSE:  219.43612387604796
100 MAE:  83.54101566671655 RMSE:  219.29771926129163
500 MAE:  83.49421501990916 RMSE:  219.04056245868955


### Gradient Boosting

In [11]:
from sklearn.ensemble import GradientBoostingRegressor

for i in num:
    gb = GradientBoostingRegressor(n_estimators=i, max_depth=10)
    gb.fit(X_train, y_train)
    y_pred = gb.predict(X_test)
    print(i, 'MAE: ', mean_absolute_error(y_val, gb.predict(X_val)), 'RMSE: ', np.sqrt(mean_squared_error(y_val, gb.predict(X_val))))

10 MAE:  111.89847780192999 RMSE:  241.85184485987278
20 MAE:  83.31872583391332 RMSE:  208.20314091300963
50 MAE:  65.42788518655746 RMSE:  184.35265157302962
100 MAE:  62.5773959022916 RMSE:  179.26642961285975
500 MAE:  64.59242206012756 RMSE:  181.2117654936575


In [13]:
from sklearn.ensemble import BaggingRegressor

for i in num:
    bag = BaggingRegressor(n_estimators=i)
    bag.fit(X_train, y_train)
    y_pred = bag.predict(X_test)
    print(i, 'MAE: ', mean_absolute_error(y_val, bag.predict(X_val)), 'RMSE: ', np.sqrt(mean_squared_error(y_val, bag.predict(X_val))))

10 MAE:  67.93791992636906 RMSE:  199.93453275898491
20 MAE:  66.21889093419236 RMSE:  193.76996285085403
50 MAE:  65.20360446387481 RMSE:  189.6819643244734
100 MAE:  65.31693856419695 RMSE:  190.94807931270782
500 MAE:  64.62492326277037 RMSE:  190.07366415124457


In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
print('MAE: ', mean_absolute_error(y_val, xgb_model.predict(X_val)), 'RMSE: ', np.sqrt(mean_squared_error(y_val, xgb_model.predict(X_val))))

### Any

In [ ]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('rf', RandomForestRegressor(n_estimators=100, max_depth=10)),
    ('gb', GradientBoostingRegressor(n_estimators=100, max_depth=10)),
    ('ada', AdaBoostRegressor(n_estimators=100)),
    ('bag', BaggingRegressor(n_estimators=100)),
    ('et', ExtraTreesRegressor(n_estimators=100))
]

stack = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stack.fit(X_train, y_train)
y_pred = stack.predict(X_test)
print('MAE: ', mean_absolute_error(y_val, stack.predict(X_val)), 'RMSE: ', np.sqrt(mean_squared_error(y_val, stack.predict(X_val))))

### Final submission

In [ ]:
# gb = GradientBoostingRegressor(n_estimators=100, max_depth=10)
# gb.fit(X_train, y_train)
# y_pred = gb.predict(X_test)

# result['Demand'] = y_pred
# result.to_csv('submission.csv', index=False)

# result

,id,Demand
0,0,590.082373
1,1,141.536053
2,2,89.348128
3,3,89.348128
4,4,89.348128
...,...,...
5965,5965,49.498894
5966,5966,49.498894
5967,5967,49.498894
5968,5968,49.498894
